### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import pandas as pd

In [25]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [26]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, 
                                school_data, 
                                how="left", 
                                on=["school_name", "school_name"])
school_data_complete.head(3)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635


In [27]:
# global vars
# We can use these later by calling 'f_comma' or 'f_percent'
f_comma='{:,}'.format
f_percent='{:,.2f}%'.format
f_money='${:,.2f}'.format

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [28]:
# Gets the passing percent for the sent in list of columns
def get_passing_percent(data, scores, passing_score=70.0):
    passing = pd.DataFrame()
    # Stores all the scores as a dataFrame of bools If they pass
    for score in scores:
        passing[score] = data[score] >= passing_score
    # returns a single column with the ALL boolean test (they must all be true)
    return passing.all(axis='columns').value_counts(normalize=True)[True] * 100

# Get the scores of the sent in school
def get_scores_summary(data):
    columns=['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading','% Overall Passing']
    summary = pd.DataFrame({columns[0]: [data['math_score'].mean()],
                            columns[1]: data['reading_score'].mean(),
                            columns[2]: get_passing_percent(data, ['math_score']),
                            columns[3]: get_passing_percent(data, ['reading_score']),
                            columns[4]: get_passing_percent(data, ['math_score', 'reading_score']) })
    # format columns
    summary[columns[2]] = summary[columns[2]].map(f_percent)
    summary[columns[3]] = summary[columns[3]].map(f_percent)
    summary[columns[4]] = summary[columns[4]].map(f_percent)
    return summary

# get the sumary for the district
def district_summary(data):
    columns=['Total Schools', 'Total Students', 'Total Budget']
    summary = pd.DataFrame({columns[0]: [len(data['school_name'].unique())],
                            columns[1]: data['student_name'].count(),
                            columns[2]: data.groupby(['school_name'])['budget'].max().sum() })
    summary[columns[0]] = summary[columns[0]].map(f_comma)
    summary[columns[1]] = summary[columns[1]].map(f_comma)
    summary[columns[2]] = summary[columns[2]].map(f_money)
    return summary.join(get_scores_summary(data))

district_summary(school_data_complete)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [29]:
# Get the schools summary
def get_schools_summary(data):
    summary = pd.DataFrame()
    columns = ['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget']
    for name, group in data.groupby('school_name'):
        school = pd.DataFrame({columns[0]: [group['type'].iloc[0]],
                               columns[1]: group['Student ID'].count(),
                               columns[2]: group['budget'].iloc[0] })
        school[columns[3]] = school[columns[2]] / school[columns[1]]
        # format
        school[columns[1]] = school[columns[1]].map(f_comma)
        school[columns[2]] = school[columns[2]].map(f_money)
        school[columns[3]] = school[columns[3]].map(f_money)
        # add scores
        school = school.join(get_scores_summary(group))
        school.index=[name]
        summary = pd.concat([summary, school])
        
    return summary

schools_summary = get_schools_summary(school_data_complete)
schools_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [30]:
# Sort and display the school summary
schools_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [31]:
# Sort and display the school summary
schools_summary.sort_values(by='% Overall Passing', ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [32]:
# Create the lables to lable the grades
grade_labels = ['9th', '10th', '11th', '12th']

def scores_by_grade(data, score_type):
    grouped = pd.DataFrame(data).groupby(['school_name', 'grade'])
    scores = grouped[score_type].mean().unstack()
    scores = scores.reindex(columns=grade_labels)
    scores = scores.rename_axis(None, axis = 0)
    scores = scores.rename_axis(None, axis = 1)
    return scores

scores_by_grade(school_data_complete, 'reading_score')

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [33]:
scores_by_grade(school_data_complete, 'math_score')

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [34]:
# Get the scores summary in the sent in data by the selected column
def scores_summary_by_column(data, column_name, row_title):
    scores = pd.DataFrame()
    for name, group in data.groupby([column_name]):
        school_column = get_scores_summary(group)        
        school_column.insert(0, row_title, name)        
        scores = pd.concat([scores, school_column])
    return scores.set_index(row_title)

# split the school by student spending
def get_schools_split_by_student_spending(data, bins, labels):
    column = 'Per Student Budget'
    spending_ranges = pd.DataFrame(get_schools_summary(data)[column])
    spending_ranges[column] = spending_ranges[column].str.replace('$', '')
    spending_ranges[column] = spending_ranges[column].apply(float)
    spending_ranges[column] = pd.cut(spending_ranges[column], bins=bins, labels=labels)
    spending_ranges['school_name'] = spending_ranges.index
    return spending_ranges.reset_index(drop=True)

# create the spending bin and lable for the student spending split
spending_bins = [0,585,630,645,680]
spending_labels = ['<$585','$585-630','$630-645','$645-680']
# Split the student spending
student_spending = get_schools_split_by_student_spending(school_data_complete,
                                                         spending_bins,
                                                         spending_labels)
# merge and display school data split by spending
scores_summary_by_column(pd.merge(school_data_complete, student_spending),
                         'Per Student Budget',
                         'Spending Ranges (Per Student)').reindex(spending_labels)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.70%,96.69%,90.64%
$585-630,79.982873,82.312643,79.11%,88.51%,70.94%
$630-645,77.821056,81.301007,70.62%,82.60%,58.84%
$645-680,77.049297,81.005604,66.23%,81.11%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [35]:
# Split the school by the size
def get_schools_split_by_sizes(data, bins, labels):
    column = 'size'
    sizes = pd.DataFrame(data[['school_name', column]]).drop_duplicates()
    sizes[column] = pd.cut(sizes[column], bins=bins, labels=labels)
    sizes = sizes.rename(columns={column: 'School Size'})
    return sizes.reset_index(drop=True)

# create the spending bin and lable for the student spending split
size_bins = [0,1000,2000,5000]
size_labels = ['Small (<1,000)','Medium (1,000-2,000)','Large (2,000-5,000)']
# Split the school by sizes
school_sizes = get_schools_split_by_sizes(school_data_complete, size_bins, size_labels)
# merge and display the school split by sizes
scores_by_column(pd.merge(school_data_complete, school_sizes), 
                  'School Size', 
                  'School Size').reindex(size_labels)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
"Small (<1,000)",83.828654,83.974082,93.95%,96.04%,90.14%
"Medium (1,000-2,000)",83.372682,83.867989,93.62%,96.77%,90.62%
"Large (2,000-5,000)",77.477597,81.198674,68.65%,82.13%,56.57%


## Scores by School Type

* Perform the same operations as above, based on school type

In [36]:
# display the school split by School Type
scores_by_column(school_data_complete, 'type', 'School Type')

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.70%,96.65%,90.56%
District,76.987026,80.962485,66.52%,80.91%,53.70%
